# Librairie

In [39]:

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from glob import glob

# Fonction

In [67]:
def get_route_viz(IMO):
    '''return a plotly figure showing the vessel route, center on the centroide of coordinates'''
    fig = go.Figure()
    fig.add_traces(go.Scattergeo(
    mode = "markers+lines",
    lon =df[df["IMO"]==IMO]["Longitude"],
    lat = df[df["IMO"]==IMO]["Latitude"],
    text=df[df["IMO"]==IMO]["Time of latest position"]),
    )
    fig.update_geos(fitbounds="locations")
    return fig

# Chargement

In [3]:
csv_file_list = glob('*.csv')

In [4]:
df1 = pd.read_csv(csv_file_list[0])

In [5]:
df2 = pd.read_csv(csv_file_list[1],header=None)

In [6]:
df2.columns = df1.columns

In [7]:
df = pd.concat([df1,df2])

In [8]:
df = df[df['IMO']!=0]

# EDA

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 951 entries, 0 to 813
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                951 non-null    object 
 1   ship_name                945 non-null    object 
 2   IMO                      951 non-null    int64  
 3   Time of latest position  945 non-null    object 
 4   Latitude                 945 non-null    float64
 5   Longitude                945 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 52.0+ KB


In [10]:
df['timestamp'] = pd.to_datetime(df["timestamp"])
df['Time of latest position'] = pd.to_datetime(df["Time of latest position"])

In [11]:
dft = df[df.columns[:-2]].groupby(['IMO','ship_name']).nunique().reset_index()
dft.columns = ["IMO","ship_name","N of scrap run", "N of different latest position"]

In [83]:
dft

,IMO,ship_name,N of scrap run,N of different latest position
0,8028412,ANNIE HILLINA,46,5
1,8209171,DIRK DIRK,45,27
2,8224406,PRINS BERNHARD,43,43
3,8224418,ALIDA,48,42
4,8301187,MARGIRIS,44,40
5,8707446,TSARITSA,45,1
6,8707537,CORNELIS VROLIJK,45,38
7,8707745,NIDA,42,42
8,8716928,GERDA MARIA,45,45
9,8901913,ZEELAND,42,4


## Bateau OK

In [81]:
dft[dft["N of scrap run"]*0.75 <=dft["N of different latest position"]]

,IMO,ship_name,N of scrap run,N of different latest position
2,8224406,PRINS BERNHARD,43,43
3,8224418,ALIDA,48,42
4,8301187,MARGIRIS,44,40
6,8707537,CORNELIS VROLIJK,45,38
7,8707745,NIDA,42,42
8,8716928,GERDA MARIA,45,45
11,9074951,FRANK BONEFAAS,44,44
15,9187306,WILLEM VAN DER ZWAN,43,43
17,9249556,WIRON5.H1100,42,42
18,9249568,WIRON6H2200,42,42


In [13]:
liste_bateau_OK = dft[dft["N of scrap run"]==dft["N of different latest position"]]["IMO"].tolist()

In [14]:
len(liste_bateau_OK)

9

Sur les 23 bateaux suivis seuls **9** renvoient un timestamp différent de dernière position à chaque run de scraping

In [57]:
df[df["IMO"]==liste_bateau_OK[3]]

,timestamp,ship_name,IMO,Time of latest position,Latitude,Longitude
24,2023-01-05 15:38:00,FRANK BONEFAAS,9074951,2023-01-05 13:01:00+00:00,55.97903,-1.247670
52,2023-01-05 17:07:00,FRANK BONEFAAS,9074951,2023-01-05 16:08:00+00:00,56.82451,-1.220873
67,2023-01-05 17:21:00,FRANK BONEFAAS,9074951,2023-01-05 16:21:00+00:00,56.88923,-1.220090
79,2023-01-05 18:17:00,FRANK BONEFAAS,9074951,2023-01-05 17:17:00+00:00,57.15718,-1.252282
101,2023-01-05 18:37:00,FRANK BONEFAAS,9074951,2023-01-05 17:37:00+00:00,57.25628,-1.264957
123,2023-01-06 16:06:00,FRANK BONEFAAS,9074951,2023-01-06 15:07:00+00:00,59.98968,-2.980707
145,2023-01-06 16:26:00,FRANK BONEFAAS,9074951,2023-01-06 15:27:00+00:00,59.99835,-2.968552
167,2023-01-06 16:40:00,FRANK BONEFAAS,9074951,2023-01-06 15:41:00+00:00,60.01567,-3.011442
7,2023-01-10 21:22:00,FRANK BONEFAAS,9074951,2023-01-10 21:21:00+00:00,60.90185,-0.726798
29,2023-01-10 21:37:00,FRANK BONEFAAS,9074951,2023-01-10 21:37:00+00:00,60.89074,-0.668402


In [53]:
liste_bateau_OK[0]

8224406

In [68]:
get_route_viz(8224406)

In [16]:
dft.sample(5)

,IMO,ship_name,N of scrap run,N of different latest position
1,8209171,DIRK DIRK,45,27
7,8707745,NIDA,42,42
5,8707446,TSARITSA,45,1
19,9690688,MARK,44,44
0,8028412,ANNIE HILLINA,46,5


## bateau KO

In [17]:
liste_bateau_KO = dft[dft["N of different latest position"]
                      < 6]["IMO"].tolist()

In [19]:
liste_bateau_KO

[8028412, 8707446, 8901913, 8918318, 9182801, 9204556]

Certains bateaux ne ping quasi pas sur la période, nous avons 6 bateaux qui ping 5 fois ou moins sur 42 + run de scrap

In [88]:
df[df["IMO"] == 8901913].drop_duplicates(subset=["Time of latest position"])

,timestamp,ship_name,IMO,Time of latest position,Latitude,Longitude
38,2023-01-05 15:38:00,ZEELAND,8901913,2023-01-05 14:42:00+00:00,59.84293,-3.265733
93,2023-01-05 18:17:00,ZEELAND,8901913,2023-01-05 17:10:00+00:00,59.95162,-3.681865
137,2023-01-06 16:06:00,ZEELAND,8901913,2023-01-06 09:23:00+00:00,59.93474,-5.615383
21,2023-01-10 21:22:00,ZEELAND,8901913,2023-01-07 09:35:00+00:00,58.91524,-8.373272
549,2023-01-11 03:15:00,NaN,8901913,NaT,NaN,NaN


 - ALINA (8918318) -> ping au port sur toute la période
 - ANNELIES ILENA (9204556) -> Au port time of latest position ne change pas 
 

In [85]:
for imo in liste_bateau_KO:
    print(imo)
    get_route_viz(imo).show()

8028412


8707446


8901913


8918318


9182801


9204556


## Bateau arret ponctuel

In [91]:
liste_bateau_partiel = dft[(dft["N of scrap run"]*0.75 <= dft["N of different latest position"])
                           & (dft["N of scrap run"] != dft["N of different latest position"])]["IMO"].tolist()

In [98]:
df[df["IMO"] == liste_bateau_partiel[1]].groupby([ "Time of latest position"]).agg({
    "Latitude": lambda x: len(list(x)), "Longitude": lambda x: len(list(x))})

,Latitude,Longitude
Time of latest position,,
2023-01-05 10:06:00+00:00,4,4
2023-01-06 10:40:00+00:00,1,1
2023-01-06 15:08:00+00:00,2,2
2023-01-10 21:21:00+00:00,1,1
2023-01-10 21:34:00+00:00,1,1
2023-01-10 21:51:00+00:00,1,1
2023-01-10 21:55:00+00:00,1,1
2023-01-10 22:16:00+00:00,1,1
2023-01-10 22:28:00+00:00,1,1


In [96]:
get_route_viz(liste_bateau_partiel[0])

In [97]:
get_route_viz(liste_bateau_partiel[1])

In [99]:
get_route_viz(liste_bateau_partiel[2])